In [1]:
import pandas as pd
import urllib.parse
from random import shuffle
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [2]:
mt = pd.read_csv("meta.csv")

In [3]:
# mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
tags = list(zip(mt["id"], mt["Singer"], mt["Album_Name"]))
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
cols = ['id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages', 'Ranked']
df = pd.DataFrame(columns=cols)

In [4]:
lower_alnum_dash = lambda s: "".join(x for x in s.replace(" ", "-").lower() if (x.isalnum() or x == "-" ))
tags = [('stevereich2018pultet', 'Steve Reich', 'Pulse/Quartet'),
 ('christinavantzou2018no4no4', 'Christina Vantzou', 'No. 4'),
 ('billcallahan2022ytiaer', 'Bill Callahan', 'YTILAER')]

max_iter = 1 # one iteration is enough for most cases, raise it if errors occur
iter = 0
while iter < max_iter:
  new_tags = []
  for uid, singer, album in tags:
    # Remove dashes first
    singer = singer.replace("-", " ")
    album = album.replace("-", " ")
    try:
      # Try url first
      url = "https://rateyourmusic.com/release/album/%s/%s/" % (lower_alnum_dash(singer), lower_alnum_dash(album))
      tag = singer + " - " + album
      album_infos = network.get_album_infos(url=url)
    except:
      album_infos = []
    
    if not album_infos:
      try:
        # Try search then
        url_tag = urllib.parse.quote(tag)
        album_infos = network.get_album_infos(name=url_tag) # get album by search
      except IndexError as e:
        print("❌ ", tag)
      
    if not album_infos:
      new_tags.append((uid, singer, album))
    else:
      print("✅ ", tag)
      album_infos["id"] = uid
      df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    # except (IndexError, AttributeError):

  tags = new_tags
  iter += 1

❌  Steve Reich - Pulse/Quartet
✅  Christina Vantzou - No. 4
✅  Bill Callahan - YTILAER


In [5]:
tags

[('stevereich2018pultet', 'Steve Reich', 'Pulse/Quartet')]

In [6]:
xdf = df.reset_index()
xdf = df[cols]
xdf.reset_index()
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages,Ranked
count,2,2,2,2,2,2,2,1,2,2,0,2
unique,2,2,2,2,2,2,2,1,2,2,0,2
top,christinavantzou2018no4no4,Nº4,Christina Vantzou,6 April 2018,3.32\n \n/ 5.00.5\n\n ...,"Ambient, Drone\nModern Classical, Dark Ambient","dark, minimalistic, atmospheric, sombre",English,"[Glissando for Bodies and Machines in Space, P...","[#7a7a7a, #cccccc]",NaN,#674 for 2018
freq,1,1,1,1,1,1,1,1,1,1,NaN,1


In [7]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/opt/homebrew/anaconda3/envs/edav/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
xdf = xdf[['id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme','Ranked']]
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked
count,2,2,2,2,2,2,2,1,2,2,2
unique,2,2,2,2,2,2,2,1,2,2,2
top,christinavantzou2018no4no4,Nº4,Christina Vantzou,6 April 2018,3.32\n \n/ 5.00.5\n\n ...,"Ambient, Drone\nModern Classical, Dark Ambient","dark, minimalistic, atmospheric, sombre",English,"[Glissando for Bodies and Machines in Space, P...","[#7a7a7a, #cccccc]",#674 for 2018
freq,1,1,1,1,1,1,1,1,1,1,1


In [9]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [10]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [11]:
xdf

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked,Num_of_RYM_Ratings
0,christinavantzou2018no4no4,Nº4,Christina Vantzou,6 April 2018,3.32,"Ambient, Drone\nModern Classical, Dark Ambient","dark, minimalistic, atmospheric, sombre",NaN,"[Glissando for Bodies and Machines in Space, P...","[#7a7a7a, #cccccc]",#674 for 2018,378
0,billcallahan2022ytiaer,YTI⅃AƎЯ,Bill Callahan,14 October 2022,3.57,"Americana, Singer-Songwriter\nAlt-Country, Fol...","male vocals, introspective, pastoral, poeti...",English,"[First Bird, Everyway, Bowevil, Partition, Lil...","[#a0a5a1, #324856, #4f705f, #734144]",#104 for 2022,895


In [12]:
xdf.to_csv("rym_add.csv")